## Imports

In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))
print(sys.path[-1]) 


import rasterio
import numpy as np
from Ortomosaicos import load_orthomosaics, list_orthomosaics, show_orthomosaic

## Lista de archivos

In [ ]:
list_orthomosaics("10ene")
list_orthomosaics("17ene")
list_orthomosaics("24ene")

## EDA

### Carga de ortomosaicos

In [ ]:
ortomosaicos_10 = load_orthomosaics("10ene")
ortomosaicos_17 = load_orthomosaics("17ene")
ortomosaicos_24 = load_orthomosaics("24ene")

datasets = {
    "10ene": ortomosaicos_10,
    "17ene": ortomosaicos_17,
    "24ene": ortomosaicos_24
}

# Tipos de ortomosaicos a comparar
tipos = ["rgb", "red", "nir", "ms"]

### Función de extracción de metadatos

In [ ]:
# ### Función auxiliar: extrae metadatos (forma, canales, tipo de dato)

def get_metadata(path_or_array):
    """
    Devuelve un diccionario con (shape, canales, dtype) de un ortomosaico.
    """
    if isinstance(path_or_array, str) and os.path.exists(path_or_array):
        with rasterio.open(path_or_array) as src:
            shape = (src.height, src.width)
            count = src.count
            dtype = src.dtypes[0]
    elif isinstance(path_or_array, np.ndarray):
        count, height, width = path_or_array.shape
        shape = (height, width)
        dtype = path_or_array.dtype
    else:
        return None
    return {"shape": shape, "channels": count, "dtype": dtype}


### Comparación de metadatos por tipo

In [ ]:
# %% [markdown]
# ### Comparación de metadatos por tipo de ortomosaico

# %%
from pprint import pprint

for tipo in tipos:
    print(f"\n🔹 {tipo.upper()} — comparación entre campañas")
    info = {}
    for fecha, dataset in datasets.items():
        if dataset[tipo] is not None:
            info[fecha] = get_metadata(dataset[tipo])
        else:
            info[fecha] = "❌ No encontrado"
    pprint(info)


## Normalización

### Normalización espacial

Para asegurar comparabilidad entre campañas, se adopta como referencia espacial el **ortomosaico multiespectral del 17 de enero**.  
Aunque posee menor resolución (menos píxeles) que los RGB, es el producto **más estable y calibrado radiométricamente**, lo que garantiza coherencia física entre fechas.  

Las demás capas (RGB, NIR, RED, MS de otras fechas) se re-muestrearán a su misma grilla, resolución y sistema de coordenadas.

El ortomosaico está georreferenciado en metros, dentro de la proyección UTM 21 Sur.
Gracias a eso, los píxeles se alinean correctamente entre campañas y se pueden superponer con otras capas (como parcelas, DSM, etc.).


#### Referencia espacial (MS 17/ene)

Qué hace: abre el multiespectral del 17 de enero y guarda su perfil (CRS, transform, width/height).
Por qué: esa será la “grilla” a la que vamos a llevar todo lo demás.

In [ ]:
# Paso 0.1 — Obtener la RUTA del MS (17/ene) y extraer la "grilla" de referencia
# --------------------------------------------------------------------------------
# Objetivo:
# - Conseguir la ruta (path) del ortomosaico multiespectral del 17 de enero.
# - Abrirlo con rasterio y guardar su perfil (CRS, transform, ancho, alto).
# - Esta "grilla" será la referencia para alinear todas las demás imágenes.

# 1) Pedimos a nuestro loader las RUTAS de los archivos de la fecha 17/ene.
#    NOTA: usamos load_arrays=False para que NO cargue arrays en memoria,
#          sino que devuelva un diccionario con rutas a cada ortomosaico.
paths_17 = load_orthomosaics("17ene", load_arrays=False)

# 2) Tomamos específicamente la ruta del multiespectral ("ms").
#    Si la clave no existe o la ruta no es válida, interrumpimos con un assert.
ref_path = paths_17["ms"]
assert isinstance(ref_path, str) and os.path.exists(ref_path), \
       "No encuentro la ruta del MS 17/ene."

# 3) Abrimos el archivo con rasterio para leer metadatos de la grilla.
#    - profile: incluye crs, transform, width, height, dtype, count, etc.
#    - crs: sistema de referencia de coordenadas (ej. EPSG:32721)
#    - transform: mapea índices de píxel a coordenadas del mundo real
#    - width/height: dimensiones en píxeles (ancho/alto)
with rasterio.open(ref_path) as ref_src:
    REF_PROFILE   = ref_src.profile.copy()   # Copia del perfil completo del raster
    REF_CRS       = ref_src.crs              # CRS de la imagen (proyección/datum)
    REF_TRANSFORM = ref_src.transform        # Transformación afín (píxel -> coordenadas)
    REF_WIDTH     = ref_src.width            # Ancho (px) de la referencia
    REF_HEIGHT    = ref_src.height           # Alto (px) de la referencia

# 4) Mensaje informativo: confirmamos qué archivo usamos y su georreferenciación.
print("Referencia espacial:", os.path.basename(ref_path))
print(f"CRS: {REF_CRS}\nDims: {REF_HEIGHT} x {REF_WIDTH}")


#### Alineación espacial

In [ ]:
# Paso 0.2 — Alinear todos los ortomosaicos a la grilla del MS 17/ene
# -----------------------------------------------------------------------------
# Objetivo:
#  - Garantizar que todos los ortomosaicos (10ene, 17ene, 24ene) tengan
#    el mismo tamaño, resolución, sistema de coordenadas (CRS) y transform.
#  - Esto permite comparar píxel a píxel las distintas fechas.
#  - Usamos como "plantilla" la grilla definida en REF_PROFILE (del MS 17/ene).

# Importamos las herramientas necesarias de rasterio:
# - WarpedVRT: permite crear una vista "virtual" reproyectada sin modificar el archivo.
# - Resampling: define el método de interpolación (bilinear, nearest, etc.).
from rasterio.vrt import WarpedVRT
from rasterio.enums import Resampling


def align_to_reference(path_tif: str, ref_profile: dict,
                       resampling: Resampling = Resampling.bilinear) -> np.ndarray:
    """
    Re-muestrea un ortomosaico para alinearlo a la grilla de referencia.

    Parámetros
    ----------
    path_tif : str
        Ruta al archivo .tif que se quiere alinear.
    ref_profile : dict
        Perfil de referencia (REF_PROFILE) obtenido del MS 17/ene.
    resampling : rasterio.enums.Resampling
        Método de re-muestreo. 'bilinear' es apropiado para datos continuos (imágenes).

    Retorna
    -------
    np.ndarray
        Array (bandas, alto, ancho) alineado al sistema de referencia.
    """
    # Si la ruta no existe, devolvemos None para no interrumpir el proceso.
    if path_tif is None or not os.path.exists(path_tif):
        return None

    # Abrimos el raster original
    with rasterio.open(path_tif) as src:
        # Definimos las opciones del VRT (vista reproyectada temporal):
        # - CRS, transform, ancho y alto iguales a los de la referencia
        # - Método de remuestreo: bilinear
        vrt_opts = dict(
            crs=ref_profile["crs"],
            transform=ref_profile["transform"],
            width=ref_profile["width"],
            height=ref_profile["height"],
            resampling=resampling
        )

        # Creamos la vista virtual (sin guardar archivo intermedio)
        with WarpedVRT(src, **vrt_opts) as vrt:
            # Leemos los datos ya re-muestreados en memoria (float32)
            data = vrt.read(out_dtype="float32")  # salida: (bandas, H, W)

    return data


# -----------------------------------------------------------------------------
# Construimos el diccionario 'aligned' con los ortomosaicos ya alineados.
# Cada clave principal es una fecha ('10ene', '17ene', '24ene'),
# y dentro tiene un subdiccionario con los distintos tipos ('rgb', 'red', 'nir', 'ms').
# -----------------------------------------------------------------------------

aligned = {fecha: {} for fecha in datasets.keys()}

for fecha in datasets.keys():
    # Obtenemos las rutas de esa fecha sin cargar los arrays en memoria.
    paths = load_orthomosaics(fecha, load_arrays=False)

    # Alineamos cada tipo de ortomosaico a la grilla del MS 17/ene.
    for tipo in ["rgb", "red", "nir", "ms"]:
        aligned[fecha][tipo] = align_to_reference(
            paths.get(tipo),
            REF_PROFILE,
            resampling=Resampling.bilinear
        )

# -----------------------------------------------------------------------------
# Chequeo rápido de control:
# Imprimimos la forma (bandas, alto, ancho) de cada tipo por fecha.
# Todas las fechas deben compartir el mismo alto y ancho (621 x 1090).
# -----------------------------------------------------------------------------

for tipo in ["rgb", "red", "nir", "ms"]:
    formas = {
        fecha: (None if aligned[fecha][tipo] is None else aligned[fecha][tipo].shape)
        for fecha in aligned.keys()
    }
    show_orthomosaic()
    print(f"{tipo.upper()} ->", formas)


In [ ]:
# Paso 0.3 — Visualización de ortomosaicos alineados
# -----------------------------------------------------------------------------
# Qué hace:
#  - Usa show_orthomosaic para mostrar las imágenes alineadas (ya con misma grilla).
#  - Permite verificar visualmente que todas las campañas estén correctamente superpuestas.
#  - Se recomienda revisar que el encuadre y orientación coincidan entre fechas.

for tipo in ["rgb", "red", "nir"]:
    print(f"\n🖼️ Mostrando ortomosaicos alineados: {tipo.upper()}")
    for fecha in ["10ene", "17ene", "24ene"]:
        img = aligned[fecha][tipo]
        if img is not None:
            show_orthomosaic(img, title=f"{tipo.upper()} - {fecha}")
        else:
            print(f"⚠️ No se encontró {tipo} para {fecha}")


### Normalización radiométrica

### Normalización radiométrica — Escalado de valores [0, 1]

Una vez alineadas espacialmente, las imágenes aún difieren en su **escala de valores** (radiometría):
- Los ortomosaicos RGB suelen estar en enteros de 8 bits → valores de 0 a 255.
- Los ortomosaicos multiespectrales (RED, NIR, MS) suelen usar 16 bits o flotantes → valores mucho mayores.

Para poder compararlas y combinarlas (por ejemplo, al calcular índices como NDVI o al entrenar un modelo),  
es necesario **llevar todos los valores a un rango común [0, 1]**, convirtiendo a `float32`.

Esto no altera la forma de la imagen, solo **ajusta su escala** para que todas las bandas sean coherentes entre sí.
